In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell; 
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("Combustion_HeatedSquareCylinder_Re200");

In [ ]:
static var myBatch = GetDefaultQueue();
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

In [ ]:
int[] Resolutions = new int[]{2}; 
int[] DGdegree = new int[]{3};
int[] nCells  =  Resolutions.Select(r => (int)(Math.Pow(2,r+1))).ToArray();
double[] Reynolds = new double[] { 300}; 
int dgMF = 3;
int procNum = 16;

In [ ]:
myDb.Sessions

In [ ]:
//  myDb.Sessions[10].Timesteps.Skip(1432).Where(ts=>ts.TimeStepNumber.MajorNumber %2 == 0).Export().WithSupersampling(1).Do()

## Create grid

In [ ]:
public static class GridFactory {
    public static double[] GetRNodes(int Res) { 
//        var rNodes = Grid1D.ExponentialSpaceing(0.5, 29.5, Res + 1, 1.2);
        // var rNodes = Grid1D.TanhSpacing(0.5, 29.5, Res + 1, 2, true);
        var rNodes = Grid1D.TanhSpacing(0.5, 15.5, Res + 1, 2, true);
        return rNodes;
    }
 
    static double[] GetSNodes(int Res) {
        var sNodes =  Grid1D.ExponentialSpaceing(0.0, 1.0, Res + 1, 1.0);
        return sNodes;
    }
    
    

    public static double[] getNodesOneSide(int Res, double l, double r, double factor, bool leftsideOK) {
        double[] result;
    
        if(leftsideOK){ // i want nodes closer to l
            var nodes = GenericBlas.SinLinSpacing(l, r + (r - l), factor, 2 * Res + 1).ToList();
            result = nodes.GetSubVector(0, nodes.Count / 2 + 1); // Take only "left side" of node array
        } else{ // i want nodes closer to r
            var nodes = GenericBlas.SinLinSpacing(l-(r-l),r, factor, 2 * Res + 1).ToList();
            result = nodes.GetSubVector(nodes.Count / 2 , nodes.Count / 2+1 ); // Take only "left side" of node array
        }
        return result;
    }
    
    
    
    public static double[] GetXNodes_WithCutOut(int Res) { 
        double L11 = -4.5;
        double L12 = -0.5;
        double L21 = L12;
        double L22 = 0.5;
        double L31 = L22;
        double L32 = 22.5;
    
        var xNodes1 = GenericBlas.SinLinSpacing(L11,L12, 0, 4*Res + 1).ToList();
        var xNodes2 = GenericBlas.SinLinSpacing(L21,L22, 0, 1*Res + 1).ToList();
        var xNodes3 = GenericBlas.SinLinSpacing(L31,L32, 0, 22*Res + 1).ToList();
        // var xNodes3 =getNodesOneSide(43*Res+1, L31,L32, 0.0,true); // nodes closer to L31
    
    
        var xNodes = new List<double>();
        xNodes.AddRange(xNodes1.SkipLast(1));
        xNodes.AddRange(xNodes2.SkipLast(1));
        xNodes.AddRange(xNodes3.SkipLast(0));
        return xNodes.ToArray();
    }
    public static double[] GetYNodes_WithCutOut(int Res, double eps) { 
        double L11 = -8.5 + eps;
        double L12 = -0.5+ eps*0;
        double L21 = L12;
        double L22 = 0.5+ eps*0;
        double L31 = L22;
        double L32 = 8.5+ eps;
    
        // var yNodes1 = GenericBlas.SinLinSpacing(L11,L12, 0, 15*Res + 1).ToList();
        // var yNodes2 = GenericBlas.SinLinSpacing(L21,L22, 0, 2*Res + 1).ToList();
        // var yNodes3 = GenericBlas.SinLinSpacing(L31,L32, 0, 15*Res + 1).ToList();
    
        var yNodes1 = getNodesOneSide(7*Res +1,L11,L12,0.0,false); // nodes closer to L12
        var yNodes2 = GenericBlas.SinLinSpacing(L21,L22, 0, Res + 1).ToList();
        var yNodes3 = getNodesOneSide(7*Res +1,L31,L32,0.0,true); // nodes closer to L31
    
        var xNodes = new List<double>();
        xNodes.AddRange(yNodes1.SkipLast(1));
        xNodes.AddRange(yNodes2.SkipLast(1));
        xNodes.AddRange(yNodes3.SkipLast(0));
        return xNodes.ToArray();
    }
    

    public static Grid2D GenerateGrid_curved(int Res, int dg) { 
        var rNodes = GetRNodes(Res);
        var sNodes = GetSNodes(Res);
        Grid2D grd;
        if(dg == 1){
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_4, true);
        } else if(dg == 2){
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_9, true);
        } else {
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_16, true);
        }
        
      
        grd.EdgeTagNames.Add(1, "Velocity_inlet");
        grd.EdgeTagNames.Add(2, "Pressure_outlet");
        grd.EdgeTagNames.Add(3, "Velocity_inlet_Cylinder");

        // grd.EdgeTagNames.Add(3, "Wall_tempfixed_hot");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            // left velocity inlet
            if ((x * x + y * y) > 2 && x <= 0)
                return 1;

            //right pressure outlet
            if ((x * x + y * y) > 2 && x >  0)
                return 2;

            else return 3;
        });
        
             
        bool force = true; 
        myDb.SaveGrid(ref grd, force);
        
        return grd;
     }
 
      
    public static Grid2D GenerateGrid(int Res) { 

        double eps = 0.2; // A small value that "moves" the system in order to make it non symmetrical in the y direction
        double[] CutOut1Point1 = new double[2] { -0.5, -0.5 + eps*0 }; 
        double[] CutOut1Point2 = new double[2] {  0.5,  0.5 + eps*0 };
        
        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);
        
        // var xNodes = GetXNodes(Res);
        // var yNodes = GetYNodes(Res);
        var xNodes = GetXNodes_WithCutOut(Res);
        var yNodes = GetYNodes_WithCutOut(Res, eps);
        
        var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);
        
        grd.EdgeTagNames.Add(1, "Wall_tempfixed_cold");
        grd.EdgeTagNames.Add(2, "Velocity_inlet");
        grd.EdgeTagNames.Add(3, "Pressure_outlet");
        grd.EdgeTagNames.Add(4, "Velocity_inlet_Cylinder");
//        grd.EdgeTagNames.Add(5, "NoSlipNeumann");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //right pressure outlet 
            if (Math.Abs(x - 22.5 ) < 1e-8)
                return 3;

            //top wall
            if (Math.Abs(y - (8.5+ eps)  ) < 1e-8)
                // return 1;
                return 3;


            // left velocity inlet
            if (Math.Abs(x + 4.5 ) < 1e-8)
                return 2;

            //bottom Wall
            if (Math.Abs(y -(-8.5+eps) ) < 1e-8)
            // return 1;
            return 3;
            else return 4;
        });
        
        bool force = true; 
        myDb.SaveGrid(ref grd, force);
        
        return grd;
     }
 }

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double Th, double Tc, double Froude, double const_val ) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");
           stw.WriteLine("  static public double VelX(double[] X) {");
           stw.WriteLine("    return 6.0 * (0.5 + X[1]) * (0.5 - X[1]);");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double VelY(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("static public double CylinderTemperature(double[] X, double t) {");
           stw.WriteLine("double T;");
           stw.WriteLine("double TH = 1.0;");
           stw.WriteLine("double TM ="+const_val+ " ;");
           stw.WriteLine("double t1 = 2; ");// time where temperature starts increasing
           stw.WriteLine("double t2 = 5; ");// temperature peak
           stw.WriteLine("double t2_2 = 7; ");// temperature peak
           stw.WriteLine("double t3 = 10; ");// time where temperature is again at initial state
           stw.WriteLine("if (t > t1 && t < t2)");
           stw.WriteLine("T = ((TM - TH) / (t2 - t1)) * (t - t1) + TH;");
           stw.WriteLine("else if (t >= t2 && t < t2_2) {");
           stw.WriteLine("T = TM;");
           stw.WriteLine("} else if (t >= t2_2 && t < t3) {");
           stw.WriteLine("T = ((TM - TH) / (t2 - t3)) * (t - t3) + TH;");
           stw.WriteLine("} else {");
           stw.WriteLine("T = TH;");
           stw.WriteLine("}");
           stw.WriteLine("return T;");
           stw.WriteLine("}");
           

           stw.WriteLine("  static public double CylinderVx(double[] X) {");
           stw.WriteLine("    return X[0]*0.2/0.5;"); // 0.3 hardcoded velocity, 0.5 hardcoded radius
           stw.WriteLine("  }");

           stw.WriteLine("  static public double CylinderVy(double[] X) {");
           stw.WriteLine("    return X[1]*0.2/0.5;"); // 0.3 hardcoded velocity, 0.5 hardcoded radius
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureHot(double[] X) {");
           stw.WriteLine("    return 1.5;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureCold(double[] X) {");
           stw.WriteLine("    return 1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double One(double[] X) {");
           stw.WriteLine("    return 1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double Zero(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");
        
           stw.WriteLine("  static public double Temperature(double[] X) {");
           stw.WriteLine("    return (-2.333 * X[1]) + (4.333 / 2);");
           stw.WriteLine("  }");
            
        //    stw.WriteLine(" static public double InitialVelocityVx(double[] X) { ");
        //    stw.WriteLine("    return 1.0 - 1 * Convert.ToInt32(((X[0]+2)*(X[0]+2)+X[1]*X[1])<=1.0) * X[1];");
        //    stw.WriteLine("    }"); 
            
        //    stw.WriteLine(" static public double InitialVelocityVy(double[] X) { ");
        //    stw.WriteLine("    return 0.0 + 1 * Convert.ToInt32(((X[0]+2)*(X[0]+2)+X[1]*X[1])<=0.5) * (X[0]+2);");
        //    stw.WriteLine("    }"); 


        stw.WriteLine(" static public double InitialVelocityVx(double[] X) { ");
        stw.WriteLine("    return 1.0 - 1*Math.Sin(Math.Atan2(X[1], (X[0] - "+ const_val+"))) * Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1]) * Math.Exp(0.5 * (1.0 - (Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1]) * Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1])))  );");
        stw.WriteLine("    }"); 
         
        stw.WriteLine(" static public double InitialVelocityVy(double[] X) { ");
        stw.WriteLine("    return  1*Math.Cos(Math.Atan2(X[1], (X[0] - "+ const_val+"))) * Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1]) * Math.Exp(0.5 * (1.0 - (Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1]) * Math.Sqrt((X[0] - "+ const_val+") * (X[0] - "+ const_val+") + X[1] * X[1]))));");
        stw.WriteLine("    }"); 



           stw.WriteLine("  static public double ConstantVal(double[] X) {");
           stw.WriteLine("    return " + const_val+";");
           stw.WriteLine("  }");

           stw.WriteLine(" static public double InitialPressure(double[] X) { ");
            stw.WriteLine("         return (-1)* X[1] / ("+Froude * Froude +") ;");
            stw.WriteLine("    }");
            stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
   
    static public Formula Get_VelX(double Th, double Tc , double Froude) {
        return new Formula("BoundaryValues.VelX", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude, -1));
    }

    static public Formula Get_VelY(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.VelY", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_TemperatureHot(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureHot", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }

     static public Formula Get_TemperatureCold(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureCold", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }

    static public Formula Get_One(){
        return new Formula("BoundaryValues.One", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_Zero(){
        return new Formula("BoundaryValues.Zero", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_Temperature(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.Temperature", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_InitialPressure(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.InitialPressure", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_InitialVelocityVx(double xpos){
        return new Formula("BoundaryValues.InitialVelocityVx", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,xpos));
    }
    static public Formula Get_InitialVelocityVy(double xpos){
        return new Formula("BoundaryValues.InitialVelocityVy", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,xpos));
    }
    static public Formula Get_ConstantVal(double constVal){
        return new Formula("BoundaryValues.ConstantVal", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,constVal));
    }
    static public Formula Get_CylinderVelocityX(){
        return new Formula("BoundaryValues.CylinderVx", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }

    static public Formula Get_CylinderVelocityY(){
        return new Formula("BoundaryValues.CylinderVy", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }


    static public Formula Get_Temperature_Cylinder(double maxTemperature){
        return new Formula("BoundaryValues.CylinderTemperature",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,maxTemperature));
    }
}

In [ ]:

static XNSEC_Control GiveMeTheCtrlFile(int dg, int Res, bool isMF, double Re) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4;
    C.SetDGdegree(dg);
    var nCells = (int)Math.Pow(2, Res + 1);

    bool useCurvedMesh = false;
    if (useCurvedMesh) {
        C.SetGrid(GridFactory.GenerateGrid_curved(nCells, dg));
    } else {
        C.SetGrid(GridFactory.GenerateGrid(Res));
    }


    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Dgdegree", dg));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Reynolds", Re));
    C.SessionName = "Unsteady_Combustion_HeatedCyl_NoVortex_k_HighInletVelTubeRealChemPars_01DaMult" + Res + "_DG" + dg + "_Re" + Re;


    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };

    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = 1.0;// Math.Max(uInFuel, uInAir); // m/s
    double LRef = 1;

    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;

    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    Console.WriteLine("Damkoehler Number :" + C.Damk);

    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 };
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref * 1.0; 

    double maxTemperature = C.zSt * C.TFuelInlet + (1 - C.zSt) * C.TOxInlet + C.HeatRelease * C.YFuelInlet * C.zSt;

    Console.WriteLine("Max flame temperature should be: " + maxTemperature);
    C.ChemicalReactionActive = true;

    C.smoothingFactor = 0;

    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.TimeSteppingScheme = BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.BDF3;
    double dt = 0.05;
    C.dtMax = dt;
    C.dtMin = dt;
    C.Endtime = 100;
    C.NoOfTimesteps = 15000;


    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();

    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;

    C.NonLinearSolver.ConvergenceCriterion = 1e-11;
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.MaxSolverIterations = 15;

    C.PenaltyViscMomentum = 1.0 * 1;
    C.PenaltyHeatConduction = 1.0 * 1;
    C.PhysicalParameters.IncludeConvection = true;

    C.rhoOne = false;
    C.MatParamsMode = MaterialParamsMode.Constant;

    C.UseSelfMadeTemporalOperator = true;
    C.timeDerivativeEnergyp0_OK = false;
    C.timeDerivativeConti_OK = true;

    C.Reynolds = Re;
    C.Prandtl = 0.71;
    double Fr = 11111111; //shouldnt matter

    C.Froude = Fr;
    C.HeatCapacityRatio = 1.4;
    C.T_ref_Sutherland = 600;  //Check
    //==========================
    // Initial conditions
    //==========================
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_InitialVelocityVx(-2.5));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_InitialVelocityVy(-2.5));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_Zero());

    //==========================
    // Boundary conditions
    //==========================


    C.AddBoundaryValue("Velocity_inlet", VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantVal(1.0));
    C.AddBoundaryValue("Velocity_inlet", VariableNames.VelocityY, BoundaryValueFactory.Get_Zero());

    C.AddBoundaryValue("Pressure_outlet", VariableNames.Pressure, BoundaryValueFactory.Get_Zero());

    C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityX, BoundaryValueFactory.Get_CylinderVelocityX());
    C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityY, BoundaryValueFactory.Get_CylinderVelocityY());



    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double Re){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,Re);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CylinderFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells+"VariableDensity_Sutherland11_SelfmadeTempOp_dt005_Re"+Re+"DensityTimeDerivativeOK_rep1q2243" ;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    
    // Boundary and initial conditions
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantVal(0.0));
    C_MixtureFraction.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantVal(1.0));
    C_MixtureFraction.AddBoundaryValue("Velocity_inlet", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantVal(0.0));

    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    Bounds.Add(VariableNames.MixtureFraction, new Tuple<double, double>(0.0 - 1e-2, 1.0 + 1e-2)); // Between 0 and 1 per definition
    C_MixtureFraction.VariableBounds = Bounds;

    // Boundary conditions
    C_MixtureFraction.AdaptiveMeshRefinement = true;
    
   // C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 

    
//     C_MixtureFraction.AMR_startUpSweeps = 4;
   
// //    double[] p11 = new double[2] {-1.5, -1.5};
// //    double[] p12 = new double[2] { 4.0,  1.5};   
// //    var ind1     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p11, p12);
// //    ind1.maxRefinementLevel = 2;
// //    C_MixtureFraction.activeAMRlevelIndicators.Add(ind1);
   
   
// //    double[] p21 = new double[2] {-1.0, -1.0};
// //    double[] p22 = new double[2] { 2.0,  1.0};  
// //    var ind2     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p21, p22);
// //    ind2.maxRefinementLevel = 2;
// //    C_MixtureFraction.activeAMRlevelIndicators.Add(ind2);
   
   
// //    double[] p31 = new double[2] { -0.75, -0.75};
// //    double[] p32 = new double[2] {  1.50,  0.75};  
// //    var ind3     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p31, p32);
// //    ind3.maxRefinementLevel = 2;
// //    C_MixtureFraction.activeAMRlevelIndicators.Add(ind3);


    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,4) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:

  Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
  var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, Resolutions[0],Reynolds[0] );
  string jobName       = C_MixtureFraction.SessionName;
  Console.WriteLine(jobName);
  var oneJob           = new Job(jobName, solver_MF);
  oneJob.NumberOfMPIProcs =  procNum;
  oneJob.SetControlObject(C_MixtureFraction);
    oneJob.Activate(myBatch); 



In [ ]:
// myDb.Sessions[1].Timesteps.Last().Export().WithSupersampling(2).Do()
// myDb.Sessions[1].Export().WithSupersampling(2).Do()

In [ ]:
var myDb = OpenOrCreateDatabase(@"D:\DatabasesPaper\Differentiallyheatedcavity\RayleighSweep");

In [ ]:
myDb.Sessions